In [1]:
# import pandana
# import osmnx as ox
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import geoalchemy2
import numpy as np
import networkx as nx

from tqdm.notebook import tqdm
tqdm.pandas()

import hdbscan

D:\Temp\ipykernel_17668\1661411409.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# функция для чтения того, что получили в запросе
def select_pg(sql, geom):
    return gpd.read_postgis(sql, engine, geom_col = geom, crs = 3857)

engine = create_engine('postgresql+psycopg2://kb_geo:kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h@\
db03.cluster.strlk.ru/kb_graph')


#коннект к базе данных 
conn = psycopg2.connect(
    host="db03.cluster.strlk.ru",
    database="kb_graph",
    user="kb_geo",
    password="kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h"
)

In [8]:
# необходимо, чтбоы в таблице с точками, были id графа

def tsp_trip(name, graph):

    #коннект к бд
    conn = psycopg2.connect(
    host="db03.cluster.strlk.ru",
    database="kb_graph",
    user="kb_geo",
    password="kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h")


    df = select_pg(f'select * from {name}', 'geom')
    df.dropna(subset=['owner_id', 'trip'], inplace=True)
    df_nodes = df[['owner_id', 'trip']]
    df_nodes.drop_duplicates(subset=['owner_id', 'trip'], inplace=True)
    df_nodes = df_nodes.reset_index(names = 'level')
    # df.merge(df_nodes, on = [['owner_id','nodes']])
    df = df.merge(df_nodes, on=['owner_id', 'trip'])
    df.to_postgis(f'{name}_tsp', engine, if_exists = 'replace')


    # Создание курсора для выполнения запросов
    cur = conn.cursor()

    with conn:
        with conn.cursor() as cur:
            query = f"drop table if exists {name}_trip"
            cur.execute(query)

    # Закрытие курсора и соединения
    cur.close()

    # Создание курсора для выполнения запросов
    cur = conn.cursor()

    with conn:
        with conn.cursor() as cur:

            query = f"CREATE TABLE {name}_trip (\
                level int,\
                start_vid bigint, \
                end_vid bigint, \
                node  bigint,\
                edge  bigint,\
                geom geometry);"

            cur.execute(query)

    # Закрытие курсора и соединения
    cur.close()

    # Использование параметров для корректной подстановки значения переменной в запрос
    for level in df_nodes['level']:
        print(level)
        try:
            with conn:
                with conn.cursor() as cur:
                    query = f"insert into {name}_trip (level, start_vid, end_vid, node, edge, geom)\
                                with path as\
                                    (SELECT {level} as level, array_agg(node) as node FROM\
                                    pgr_TSP($$SELECT * FROM pgr_dijkstraCostMatrix('SELECT id, source, target, cost, reverse_cost FROM {graph}',\
                                    (SELECT array_agg(node_id) FROM {name}_tsp WHERE level = {level}), directed := false) $$, randomize := false)\
                                    group by level),\
                                route as (\
                                    SELECT  start_vid , end_vid , node, edge from pgr_dijkstraVia('SELECT id, source, target, cost, reverse_cost FROM {graph}',\
                                        (SELECT node FROM path WHERE level = {level}), directed := false))\
                                select {level} as level, route.*, geom from route join {graph} on edge = id"
                    # print(index)
                                    
                    cur.execute(query)
        except:
            continue




    # Закрытие курсора и соединения
    cur.close()
    conn.close()


    return df


In [17]:
# функция разделения по временным кластерам 

def cluster(name, time):

    #подлкючаемся к базе данных
    conn = psycopg2.connect(
    host="db03.cluster.strlk.ru",
    database="kb_graph",
    user="kb_geo",
    password="kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h")


    #считываем таблицу 
    df = select_pg(f'select id, geom, in_out, owner_id, date_time, url, city_left, city_right, month, user_region, photo_region from {name}', 'geom')
    #переводим время и дату в формате времени и даты
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M:%S')
    #выделяем месяц
    df['month'] = df['date_time'].dt.month
    df['ts_d'] = 0
    #формат - количество секунд с определенной даты (время в секундах с начала эпохи Unix)
    df['ts'] = df['date_time'].progress_apply(lambda x: x.timestamp())
    #сохраняются те строки 'ts' имеет более 2 значений (берем только те строки, которые имеют две разные отметки времени)
    df = df.groupby(['owner_id']).filter(lambda x: x['ts'].count() > 2)

    #функция для группировкт объектов
    #epsilon - количество секунд, которые могут разделять один кластер 
    def groupby_cluster(X):
        db = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, cluster_selection_epsilon=1*24*60*60*1*time).fit(X) 
        return pd.DataFrame(db.labels_, index=X.index)

    #группировку данных по 'owner_id', а затем применение к сгруппированных значениям функции 'groupby_cluster' 
    # Результаты кластеризации сохраняются в новом DataFrame 'df_cluster', который содержит метки кластеров для каждой строки в исходном DataFrame 'df'.
    df_cluster = df.groupby(['owner_id'], as_index=False)[['ts', 'ts_d']].progress_apply(groupby_cluster)

    #приджойниваем номера кластеров к фотографиям 
    df = df.join(df_cluster[0]).rename(columns={0: "cluster"})
    #убираем те фотографии, где метка кластера равна -1, так как это выбросы (например, только одна фотография сделана за указанный период)
    df = df[df['cluster'] != -1]

    return df

In [58]:
# объединение двух вышестоящих функций в одну
# для этой функции должен быть связный граф. сейчас используется baikal_edge_graph_connected_all

def tsp(name, time):

    #подлкючаемся к базе данных
    conn = psycopg2.connect(
    host="db03.cluster.strlk.ru",
    database="kb_graph",
    user="kb_geo",
    password="kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h")


    #считываем таблицу 
    df = select_pg(f'select id, geom, id_gis, in_out, owner_id, date_time, url, city, month, user_region, photo_region, node_id, node_geom from {name}', 'geom')")
    #переводим время и дату в формате времени и даты
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M:%S')
    #выделяем месяц
    df['month'] = df['date_time'].dt.month
    df['ts_d'] = 0
    #формат - количество секунд с определенной даты (время в секундах с начала эпохи Unix)
    df['ts'] = df['date_time'].progress_apply(lambda x: x.timestamp())
    #сохраняются те строки 'ts' имеет более 2 значений (берем только те строки, которые имеют две разные отметки времени)
    df = df.groupby(['owner_id']).filter(lambda x: x['ts'].count() > 2)

    # db = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, cluster_selection_epsilon=1*24*60*60*1*time).fit(X)
    # hdbscan_df = pd.DataFrame(db.labels_, index=X.index)

    #функция для группировкт объектов
    #epsilon - количество секунд, которые могут разделять один кластер 
    def groupby_cluster(X):
        db = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, cluster_selection_epsilon=1*24*60*60*1*time).fit(X) 
        return pd.DataFrame(db.labels_, index=X.index)

    #группировку данных по 'owner_id', а затем применение к сгруппированных значениям функции 'groupby_cluster' 
    # Результаты кластеризации сохраняются в новом DataFrame 'df_cluster', который содержит метки кластеров для каждой строки в исходном DataFrame 'df'.
    df_cluster = df.groupby(['owner_id'], as_index=False)[['ts', 'ts_d']].progress_apply(groupby_cluster)

    #приджойниваем номера кластеров к фотографиям 
    df = df.join(df_cluster[0]).rename(columns={0: "cluster"})
    #убираем те фотографии, где метка кластера равна -1, так как это выбросы (например, только одна фотография сделана за указанный период)
    df = df[df['cluster'] != -1]

    # группируем по id пользоватнля и номеру кластера, считаем количество уникальных node id 
    count = df.groupby(['owner_id', 'cluster'])['node_id'].count()
    count = count.reset_index()
    #оставляем только тех 'owner_id', 'cluster' у которыз более двух различных узлов в маршруте 
    count_more_two = count[count['node_id'] >= 2]

    #приджойниваем полученный набор данных к фотографиям и остаются только те 'owner_id', 'cluster' у которых больше двух уникальных node_id
    df = df.merge(count_more_two, on=['owner_id', 'cluster'], how='inner', indicator=True)

    #создаем датафрэйм с колонами 'owner_id', 'cluster' и списком посещенных node_id_x
    df_nodes = df.groupby(['owner_id', 'cluster'])['node_id_x'].apply(list).reset_index()
    # df_nodes = df_nodes.reset_index()


    df = df.merge(df_nodes[['index', 'owner_id', 'cluster']], on = ['owner_id', 'cluster'], how = 'inner')
    df.to_postgis(f'{name}_tsp', engine, if_exists = 'replace')

    # Создание курсора для выполнения запросов
    cur = conn.cursor()

    # Этот код выполняет SQL-запрос на удаление таблицы с названием "{name}_route", 
    # если она существует в базе данных, подключенной к курсору "cur". 
    # Оператор "with" используется для обеспечения правильного закрытия соединения с базой данных после выполнения запроса.
    with conn:
        with conn.cursor() as cur:
            query = f"drop table if exists {name}_route"
            cur.execute(query)

    # Закрытие курсора и соединения
    cur.close()

    # Создание курсора для выполнения запросов
    cur = conn.cursor()

    with conn:
        with conn.cursor() as cur:
            # query = f"CREATE TABLE if not exists {name}_path(\
            #     index integer,\
            #     node ARRAY INTEGER)"
            # cur.execute(query)

            #создаем таблицу route в которой есть:
            # index, начальная и конечная вершины пути между точками, node - номер узла по которому проходим, edge - номер ребра по которому проходим, geom - геометрия ребра
            # df_nodes['index'] - уникальные значение index и owner id
            query = f"CREATE TABLE {name}_route(\
                index int,\
                start_vid bigint, \
                end_vid bigint, \
                node  bigint,\
                edge  bigint,\
                geom geometry);"

            cur.execute(query)

    # Закрытие курсора и соединения
    cur.close()

    # Использование параметров для корректной подстановки значения переменной в запрос
    # пробегаемся по каждому уникальному значению index 
    for index in df_nodes['index']:
        try:
            with conn:
                with conn.cursor() as cur:
                    #инсертим знаение в таблицу route 
                    query = f"insert into {name}_route\
                                with path as\
                                    -- выборка данных из таблицы, содержащей узлы графа, которые нужно обойти в заданном порядке \
                                    (SELECT {index} as index, array_agg(node) as node FROM \
                                    --выполнение алгоритма TSP с помощью функции pgr_TSP\
                                    -- Она использует матрицу стоимости, созданную с помощью функции pgr_dijkstraCostMatrix\
                                    -- для нахождения оптимального маршрута, проходящего через все узлы в заданном порядке.\
                                    pgr_TSP($$SELECT * FROM pgr_dijkstraCostMatrix('SELECT id, source, target, cost, reverse_cost FROM baikal_edge_graph_connected_all', \
                                    --выборка узлов из таблицы, которые нужно обойти в задаче TSP \
                                    (SELECT array_agg(node_id_x) FROM {name}_tsp WHERE index = {index}), directed := false) $$, randomize := false)\
                                    group by index),\
                                    edge_route as (\
                                    -- выполнение алгоритма Dijkstra с помощью функции pgr_dijkstraVia для поиска кратчайшего маршрута между каждой парой узлов.\
                                    SELECT {index} as index, start_vid , end_vid , node, edge from pgr_dijkstraVia('SELECT id, source, target, cost, reverse_cost FROM baikal_edge_graph_connected_all', \
                                        (SELECT node FROM path WHERE index = {index}), directed := false))\
                                    -- объединение  таблиц edge_route и baikal_edge_graph_connected_all для получения итогового маршрута с геометрическими данными.\
                                    select edge.*, graph.geom as geom\
                                    from edge_route as edge join baikal_edge_graph_connected_all as graph on edge = graph.id ;"
                    
                    print(index)
                                    
                    cur.execute(query)
        except:
            continue

    # Закрытие курсора и соединения
    cur.close()
    conn.close()


    return df_nodes